In [1]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import SimpleITK as sitk
import skimage.io as io
from image_detector import read_image_into_ndArray, figure_images

## 2D images (jpg, png, etc)

In [ ]:
imgfile = "/home/seonwhee/Deep_Learning/Solaris/week9/FCN-BRATS/Data_zoo/BRATS/images/training/VSD.Brain.XX.O.MR_T1c.54514_1.jpg"
img2 = read_image_into_ndArray(imgfile, PlugIn='matplotlib')

## ITK metaImage

In [ ]:
mhafile = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0001_1/VSD.Brain.XX.O.MR_Flair.54512/VSD.Brain.XX.O.MR_Flair.54512.mha"
img = read_image_into_ndArray(mhafile, PlugIn='simpleitk')

In [ ]:
mhafile2 = "/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_tcia_pat105_0001/VSD.Brain.XX.O.MR_T1.35536/VSD.Brain.XX.O.MR_T1.35536.mha"
img2 = read_image_into_ndArray(mhafile2, PlugIn='simpleitk')

#### Multiple files

In [ ]:
mhafiles = glob('/home/seonwhee/Deep_Learning/Datasets/BRATS-2/Image_Data/HG/*/*/*mha')

In [ ]:
imgList = []
for a_file in mhafiles:
    filename = a_file.split("/")[-1]
    imgInput = sitk.ReadImage(a_file)
    nda = sitk.GetArrayViewFromImage(imgInput) # Get a view of the image data as a numpy array, useful for display
    img = sitk.GetImageFromArray(nda)
    imgList.append((filename, img))
    

In [ ]:
figure_images(imgList, 80)

### T1 and T2-weighted image

In [3]:
from ipywidgets import interact, FloatSlider
from myshow import myshow, myshow3d

In [ ]:
img_T1 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1.54567/VSD.Brain.XX.O.MR_T1.54567.mha")
img_T2 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T2.54569/VSD.Brain.XX.O.MR_T2.54569.mha")
img_FLAIR = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_Flair.54566/VSD.Brain.XX.O.MR_Flair.54566.mha")
img_T1c = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1c.54568/VSD.Brain.XX.O.MR_T1c.54568.mha")
# To visualize the labels image in RGB with needs a image with 0-255 range
img_T1_255 = sitk.Cast(sitk.RescaleIntensity(img_T1), sitk.sitkUInt8)
img_T2_255 = sitk.Cast(sitk.RescaleIntensity(img_T2), sitk.sitkUInt8)
img_FLAIR_255 = sitk.Cast(sitk.RescaleIntensity(img_FLAIR), sitk.sitkUInt8)
img_T1c_255 = sitk.Cast(sitk.RescaleIntensity(img_T1c), sitk.sitkUInt8)
myshow3d(img_T1)
myshow3d(img_T1_255)

In [4]:
img_T1 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1.54567/VSD.Brain.XX.O.MR_T1.54567.mha")
img_T2 = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T2.54569/VSD.Brain.XX.O.MR_T2.54569.mha")
img_FLAIR = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_Flair.54566/VSD.Brain.XX.O.MR_Flair.54566.mha")
img_T1c = sitk.ReadImage("/home/seonwhee/Deep_Learning/Datasets/BRATS2015_Training/HGG/brats_2013_pat0010_1/VSD.Brain.XX.O.MR_T1c.54568/VSD.Brain.XX.O.MR_T1c.54568.mha")

    # 모델을 생성한다.
img_T1_Slice = img_T1[:,:,77]
img_T1c_Slice = img_T1c[:,:,77]
img_T2_Slice = img_T2[:,:,77]
img_FLAIR_Slice = img_FLAIR[:,:,77]
imgVolume = sitk.JoinSeries(img_T1_Slice, img_T1c_Slice, img_T2_Slice, img_FLAIR_Slice)
width = imgVolume.GetWidth()
height = imgVolume.GetHeight()
depth = imgVolume.GetDepth()
myshow3d(imgVolume, title=['T1', 'T1c', 'T2', 'FLAIR'])

A Jupyter Widget